# ComCam Quick Start

Contact author: Prakruth Adari\
Last verified to run: 2025-03-05\
LSST Science Piplines version: Weekly 2025_09\
Container Size: medium 

An introduction to working with ComCam data.

In [ ]:
!eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.stats as stats

import pandas as pd
from matplotlib import cm
from astropy.visualization import make_lupton_rgb
from astropy.io import fits
from astropy.table import Table
# %matplotlib widget

In [ ]:
# Familiar stack packages
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Box2D, Point2I, Point2D, Extent2I, Extent2D
from lsst.afw.image import Exposure, Image, PARENT
import lsst.sphgeom

In [ ]:
from lsst.afw.image import MultibandExposure, MultibandImage
from lsst.afw.detection import MultibandFootprint
from lsst.afw.image import MultibandExposure

In [ ]:
# from lsst.meas.algorithms import SourceDetectionTask
# from lsst.meas.extensions.scarlet import ScarletDeblendTask
# from lsst.meas.base import SingleFrameMeasurementTask
# from lsst.afw.table import SourceCatalog

# import lsst.scarlet.lite as scl
# # import scarlet

In [ ]:
import os, sys

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
rng = np.random.default_rng()

In [ ]:
arcsec = 1/60.**2

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=1, Q=10):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    fp: `lsst.afw.detection.Footprint`
        Footprint that contains the peak catalog for peaks in the image.
        If `fp` is `None` then no peak positions are plotted.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    # Exposure.getBBox() returns a Box2I, a box with integer pixel coordinates that correspond to the centers of pixels.
    # Matplotlib's `extent` argument expects to receive the coordinates of the edges of pixels, which is what
    # this Box2D (a box with floating-point coordinates) represents.
    integerPixelBBox = image[bgr[0]].getBBox()
    bbox = Box2D(integerPixelBBox)
    ax.imshow(rgb, interpolation='nearest', origin='lower', extent=(bbox.getMinX(), bbox.getMaxX(), bbox.getMinY(), bbox.getMaxY()))
    if fp is not None:
        for peak in fp.getPeaks():
            ax.plot(peak.getIx(), peak.getIy(), "bx", mew=2)

### Initialize Butler

Butler has to be loaded with a repo (usually either `/repo/main` or `/repo/embargo`) and a collection. Once a `Butler` has been created with a `repo`, you can run `butler.registry.queryCollections('collectionname')` to see available collections. The wild card token '*' is enabled which makes searching for collections quite nice! It can take a few second depending on how many collections are returned so running `butler.registry.queryCollections('*')` will take a bit.

In [ ]:
obs_repo = '/repo/main'
skymap='lsst_cells_v1'

obs_butler = Butler(obs_repo)
obs_registry = obs_butler.registry

In [ ]:
DRP_list = list(obs_registry.queryCollections('*DRP*')) # All the collections with DRP in the name

In [ ]:
# Load a recent version of DP1 for ComCam

obs_repo = '/repo/main'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235' 
# Not using the 2025_09 release since there are some issues with the processing (as of March 5th 2025).
# Can check Rubin Slack #comcam-drp-processing for any updates or which ticket to use for the weekly release

obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

obs_butler = Butler(obs_repo, collections=obs_collection)
obs_registry = obs_butler.registry

### Display a coadd

In [ ]:
# Abell360_coord =  (37.8754558, 6.9716214)
Abell360_coord = (37.86501659859067, 6.982204815599694) #BCG coord


Butler requests a `tract`, `patch`, `skymap`, and sometimes `band` when accessing data. For a RA/DEC we can find the associated `tracts` and `patches` using the code below.
The wcs can be accessed per tract and as we will see later on, per coadd!

In [ ]:
ra = Abell360_coord[0]
dec = Abell360_coord[1]
spherePoint = lsst.geom.SpherePoint(ra*lsst.geom.degrees, dec*lsst.geom.degrees)
t_skymap = obs_butler.get('skyMap', skymap='lsst_cells_v1')

tract = t_skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)
tract_id = tract.tract_id
patch_id = patch.getSequentialIndex()
skymap = 'lsst_cells_v1'
wcs = tract.getWcs()
print(f"Tract: {tract_id}, Patch: {patch_id}")

To query for the coadds we can pass these into Butler directly or ask the registry which datasets are availale and then pass those into the Butler. I prefer the latter since it avoids any formatting on our part :)

In [ ]:
deepCoaddTable = np.array(list(obs_registry.queryDatasets('deepCoadd_calexp', skymap='lsst_cells_v1', patch=patch_id, tract=tract_id)))
print(deepCoaddTable)

For Abell 360, we only have imaging in `griz` bands! You can access the actual values of each dataset using `dataset.dataId` like below to get the correct order of bands.

In [ ]:
available_bands = [dct.dataId['band'] for dct in deepCoaddTable]
bands = 'zrig'

In [ ]:
# We can pass each entry of deepCoaddTable directly into the Butler and get all coadds available. 
# We then pass the individual coadds into a MultibandExposure for our full multi-band image.
new_deep_calexps = [obs_butler.get(dct) for dct in deepCoaddTable]

coadds = MultibandExposure.fromExposures(available_bands, new_deep_calexps) # Combining coadds into one `MultibandExposure` object
# new_wcs = new_deep_calexps[bands.index('i')].getWcs() #Each calexp also has a wcs attached to the image if we ever want to look at varying tracts

In [ ]:
# Easiest way to plot is with the showRGB defined above:
showRGB(coadds.image, bgr='gri', figsize=(6,6))

If you want to zoom-in, we can define a bounding box `BBox` and apply that to the entire `coadd` and get the relevant `subset.` `BBox` is defined in terms of pixel coordinates not RA/DEC, so we will use the WCS to transform from RA/DEC to X/Y and then get the a cut-out around the BCG.

The solved wcs comes with a `skyToPixel` and `pixelToSky` function to transform between the two. Using the variant `skyToPixelArray` allows for processing of numpy objects instead of creating `LSST.geom.SpherePoint`. __NOTE__: `pixelToSkyArray` only works on __floats__ not __integers__. 

In [ ]:
# wcs.pixelToSkyArray(12, 13, degrees=True)# -- This won't work!
wcs.pixelToSkyArray(12., 13., degrees=True)

In [ ]:
bcg_coords = wcs.skyToPixelArray(Abell360_coord[0], Abell360_coord[1], degrees=True) 
print(bcg_coords) #Check out the formatting so we can unpack it properly
bcg_x = bcg_coords[0][0]
bcg_y = bcg_coords[1][0]

In [ ]:
window = 400
# sampleBBox = Box2I(Point2I(10700-frame, 9500-frame), Extent2I(63+2*frame, 87+2*frame))
sampleBBox = Box2I(Point2I((bcg_x-window),(bcg_y-window) ), Extent2I(2*window, 2*window))

In [ ]:
# We can get our subset by applying the BBox to all coadds with nice python indexing!
subset = coadds[:, sampleBBox]

In [ ]:
# Easiest way to plot is with the showRGB defined above:
showRGB(subset.image, bgr='gri', figsize=(6,6))

#### Single-bands
If you want to plot only a single band, we can access the array via `subset[band-name].image.array`. This can be plotted using your favorite plotting tool with an example below using `matplotlib`. We have to flip the x-axis via `[::-1,:]` to match the formatting above and use the `extent` parameter to get the labels on the `x` and `y` axis. `norm` can be changed to some other normalization (or removed) and `vmin`/`vmax` can be modified to change the upper and lower bound of the colormap to better see some LSB features.

In [ ]:
cmap = cm.gray # 

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10, 10))

for i, b in enumerate(bands):
    ix = i//2
    iy = i%2
    
    ax[ix, iy].imshow(subset[b].image.array[::-1, :], norm='asinh', cmap=cmap,
                      extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=1) 
    ax[ix, iy].set_title(b)

### Load catalogs

There are lots of different catalogs available (which can be found via `registry.queryDatasets`). For any catalog, the schema can be obtained by adding a '.columns' to the end of the name. It's usually a good (sometimes necessary) practice to only request the columns you will need to reduce load on the Butler. The schemas are available online at the [Schema Browser](https://sdm-schemas.lsst.io/). We will be using the `objectTable` with schema found at the [DP0.2 Schema](https://sdm-schemas.lsst.io/dp02.html#Object). The columns we want are passed to the `Butler` via the `parameters` argument with a `columns` keyword argument. If you have a typo in your requested column names, the Butler will complain!

__NOTE__: The +'.columns' for schema will not show up in the registry! You will have to run `schema = butler.get(catalog_name + '.columns', skymap=...)` to get the schema.

In [ ]:
obj_table_columns = obs_butler.get('objectTable.columns', skymap=skymap,  patch=patch_id, tract=tract_id)
print(f"There are {len(obj_table_columns)} columns")

In [ ]:
# cmodel_columns = [ot for ot in obj_table_columns if 'cModel' in ot] 

In [ ]:
ap_suffix = '_ap12Flux'
kron_suffix = '_kronFlux'
cmodel_suffix = '_cModelFlux'

flux_suffix = cmodel_suffix # Choose which flux type you want

flux_bands = [b+suffix for b in available_bands for suffix in [flux_suffix, flux_suffix+'Err']]

In [ ]:
cluster_table = obs_butler.get('objectTable', skymap='lsst_cells_v1', patch=patch_id, tract=tract_id, 
                               parameters={"columns":['coord_ra', 'coord_dec', 'refExtendedness',
                                                      'detect_isTractInner', 'merge_peak_sky', 'detect_isDeblendedSource', 'detect_isPrimary',
                                                      'parentObjectId', 'shape_xx', 'shape_xy', 'shape_yy', 'refBand', 'x', 'y', 'patch'] + flux_bands})

In [ ]:
# cluster_table

We need to apply the `detect_isPrimary` flag to remove parent-childs from blends. This is the go-to flag to remove many of the duplicates that are present in the catalog but will not work for us -- we will be using a modified version that removes the `isPatchInner` flag from the `detect_isPrimary` set of flags.

A full overview of the flags can be found on the [LSST Pipelines Deblending Flags](https://pipelines.lsst.io/modules/lsst.pipe.tasks/deblending-flags-overview.html) page. 

`detect_isPrimary = detect_isTractInner & detect_isPatchInner & ~merge_peak_sky & isDeblendedSource`
 
To remove the isPatchInner we then have:
`detect_CLUSTER = detect_isTractInner & ~merge_peak_sky & isDeblendedSource`

In [ ]:
detect_CL = cluster_table['detect_isTractInner'] & ~cluster_table['merge_peak_sky'] & cluster_table['detect_isDeblendedSource']

In [ ]:
primary_cluster = cluster_table[detect_CL]

Let's convert our flux measurements to magnitudes and write them to our DataFrame `primary_cluster`.

In [ ]:
generic_mag_dict = {}

for b in bands:
    generic_mag_dict[f'{b}_mag'] = u.nJy.to(u.ABmag, primary_cluster[f'{b}{flux_suffix}'])

In [ ]:
primary_cluster = primary_cluster.assign(**generic_mag_dict)

We can overlay the detections on our subset! We will filter the catalog using the previously defined `BBox` and then plot the detections!

In [ ]:
in_bbox_filt = sampleBBox.contains(primary_cluster['x'], primary_cluster['y'])
bbox_cat = primary_cluster[in_bbox_filt]

In [ ]:
showRGB(subset.image, figsize=(6,6))
plt.plot(bbox_cat['x'], bbox_cat['y'], alpha=.5, color='r', lw=0, marker='o', fillstyle='none')

#### Color-Magnitude Diagram

With a catalog in hand we can easily get the color-magnitude diagram and see the red sequence!

In [ ]:
g = primary_cluster['g_mag']
r = primary_cluster['r_mag']
i = primary_cluster['i_mag']
gi = g-i
ri = r-i
gr = g-r

In [ ]:
plt.plot(i, gi, '.', markersize=1)
# plt.plot(i[redflag_gi], gi[redflag_gi], 'r.', markersize=1)
plt.title("Abell 360 Red Sequence")
plt.xlabel("i")
plt.ylabel("g-i")
# plt.axvline(23.8, ls='--', color='k', alpha=0.25)
plt.xlim(15, 27)
plt.ylim(0,3)

### Other tidbits

#### Querying for formatting

Finding that `skymap='lsst_cells_v1'` came from looking at the datasets availble which can be done using `registry.queryDatasets.` This command gives you the list of datasets available but also the parameters needed when requesting from Butler. In the example below, requesting `objectTable` requires a `skymap`, `tract`, and `patch`. 

In [ ]:
example_objTables = list(obs_registry.queryDatasets('objectTable'))
print(example_objTables[0])

In this example, requesting for `deepCoadd_calexps` requires a `band`, `skymap`, `tract`, and `patch`!

In [ ]:
example_coadd = list(obs_registry.queryDatasets('deepCoadd_calexp'))
print(example_objTables[0])

#### Butler Cut-outs

We can pass in a `BBox` into butler and directly get single band cut-outs. 

In [ ]:
# coaddId = {'tract': tract_id, 'patch': patch_id, 'skymap': skymap}
# obs_butler.get('deepCoadd_calexp', dataId=coaddId, parameters={'bbox': bbox})